In [1]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.5/328.5 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.7 MB/s eta 0:00:00


In [2]:
!pip install duckduckgo_search

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 11.0 MB/s eta 0:00:00


In [4]:
from openai import AzureOpenAI
import json
import requests
import pandas as pd
from duckduckgo_search import DDGS

# Set up Azure OpenAI Service credentials
client = AzureOpenAI(
    api_key='api_key',  # Replace with your api_key
    api_version='2024-02-01',
    azure_endpoint='azure_endpoint'  # Replace with your azure_endpoint
)

# Function to extract entities from a file using LLM
def extract_entities_from_file(file_name: str, entity_type: str) -> str:
    with open(file_name, 'r') as file:
        content = file.read()

    # Prompt to extract entities of a specific type from the content
    prompt = f"""
You are a helpful assistant. Extract all entities of the type '{entity_type}' from the following text:
{content}
Return only the entities as a JSON list.
"""

    # Send request to Azure OpenAI Service
    response = client.chat.completions.create(
        model="gpt4",
        messages=[{"role": "system", "content": "You are a helpful assistant."},
                  {"role": "user", "content": prompt}]
    )

    # Extract entities from response
    entities = response.choices[0].message.content
    return entities

# Function to perform an internet search using DuckDuckGo Instant Answer API and LLM
def internet_search_attribute(a_entity: str, a_attribute: str) -> str:
    # Perform search query using DuckDuckGo
    query = f"what is {a_entity}'s {a_attribute}?"
    results = list(DDGS().text(query, max_results=1))

    if not results:
        return json.dumps({a_entity: {a_attribute: "No results found"}})

    search_result = results[0]["body"]

    # Prompt to extract the attribute from the search result
    prompt = f"""
You are a helpful assistant. Based on the search result below, provide the {a_attribute} of {a_entity}:
Search result: {search_result}
Return the result as a JSON structure of the form: {{"a_entity": {a_entity}, "{a_attribute}": Answer}}
"""

    # Send request to Azure OpenAI Service
    response = client.chat.completions.create(
        model="gpt4",
        messages=[{"role": "system", "content": "You are a helpful assistant."},
                  {"role": "user", "content": prompt}]
    )

    # Extract answer from response
    answer = response.choices[0].message.content
    return answer

# Function to generate a Python program for data analysis using LLM
def generate_analysis_program(analysis_request: str, input_file: str, columns: str, row_example: str, output_file: str) -> str:
    # Prompt to generate a Python program for analyzing a CSV file
    prompt = f"""
I need to generate a Python program to analyze a CSV file. Here are the details:
- Analysis request: {analysis_request}
- Input file: {input_file}
- Columns: {columns}
- Example row: {row_example}
- Output file: {output_file}

Please generate the Python code for this analysis. The program should read the CSV file, perform the analysis based on the request, and write the output as a JSON object to the output file.
"""

    # Send request to Azure OpenAI Service
    response = client.chat.completions.create(
        model="gpt4",
        messages=[{"role": "system", "content": "You are a helpful assistant."},
                  {"role": "user", "content": prompt}]
    )

    # Extract generated code from response
    generated_code = response.choices[0].message.content
    return clean_code(generated_code)

# Function to clean the generated code by extracting the code block
def clean_code(content):
    # Extract content between ```python and ``` markers
    try:
        # Splitting content to capture only the code between the specific markers
        start_code = content.index("```python") + len("```python")
        end_code = content.index("```", start_code)
        content = content[start_code:end_code].strip()
    except ValueError:
        "There was an error. Please try again"
    return content

# Function to execute a Python program
def execute_python_program(program_fn: str, output_fn: str) -> str:
    try:
        # Execute the Python program
        exec(open(program_fn).read())
        with open(output_fn, 'r') as file:
            return file.read()
    except Exception as e:
        return str(e)

# Function to write content to a file
def write_file(file_content: str, fn: str):
    # Write the content to the specified file
    with open(fn, "w") as file:
        file.write(file_content)
    return f"File content was successfully written to {fn}"

# Agent loop to process the input JSON and handle function calls
def agent_loop(input_json):
    query_file = input_json["query_name"]
    log_file = f"log_{query_file.split('.')[0]}.txt"
    llm_call_count = 0
    function_call_count = 0
    LLM_CALL_LIMIT = 10
    FUNCTION_CALL_LIMIT = 10

    with open(log_file, "w") as log:

        resources = input_json["file_resources"]

        with open(query_file, "r") as file:
            query = file.read()

        messages = [{"role": "user", "content": f"{query}\n resources:{resources}"}]

        # Define available functions for the agent
        functions = [
            {
                "name": "extract_entities_from_file",
                "description": "Extract entities from a text file",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "file_name": {"type": "string"},
                        "entity_type": {"type": "string"}
                    },
                    "required": ["file_name", "entity_type"]
                }
            },
            {
                "name": "internet_search_attribute",
                "description": "Search the internet for an attribute of an entity",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "a_entity": {"type": "string"},
                        "a_attribute": {"type": "string"}
                    },
                    "required": ["a_entity", "a_attribute"]
                }
            },
            {
                "name": "generate_analysis_program",
                "description": "Generate a Python program to analyze a CSV file",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "analysis_request": {"type": "string"},
                        "input_file": {"type": "string"},
                        "columns": {"type": "string"},
                        "row_example": {"type": "string"},
                        "output_file": {"type": "string"}
                    },
                    "required": ["analysis_request", "input_file", "columns", "row_example", "output_file"]
                }
            },
            {
                "name": "execute_python_program",
                "description": "Execute a Python program",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "program_fn": {"type": "string"},
                        "output_fn": {"type": "string"}
                    },
                    "required": ["program_fn", "output_fn"]
                }
            },
            {
                "name": "write_file",
                "description": "Write content to a file",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "file_content": {"type": "string"},
                        "fn": {"type": "string"}
                    },
                    "required": ["file_content", "fn"]
                }
            }
        ]

        # Function to call the appropriate function based on the function name
        def call_function(function_name, arguments):
            nonlocal function_call_count
            function_call_count += 1

            if function_call_count > FUNCTION_CALL_LIMIT:
                return "Function call limit exceeded"

            if function_name == "extract_entities_from_file":
                return extract_entities_from_file(**arguments)
            elif function_name == "internet_search_attribute":
                return internet_search_attribute(**arguments)
            elif function_name == "generate_analysis_program":
                return generate_analysis_program(**arguments)
            elif function_name == "execute_python_program":
                return execute_python_program(**arguments)
            elif function_name == "write_file":
                return write_file(**arguments)

        # Loop to process the input and handle function calls
        while True:
            if llm_call_count >= LLM_CALL_LIMIT or function_call_count >= FUNCTION_CALL_LIMIT:
                final_response = {
                    "error": "LLM or function call limit exceeded",
                    "llm_calls": llm_call_count,
                    "function_calls": function_call_count
                }
                break

            # Send request to Azure OpenAI Service
            response = client.chat.completions.create(
                model="gpt4",
                messages=messages,
                functions=functions,
                function_call="auto"
            )

            llm_call_count += 1

            response_message = response.choices[0].message
            function_call = response_message.function_call

            if function_call:
                function_name = function_call.name
                function_args = json.loads(function_call.arguments)

                log.write(f"**Entering agent {function_name}**\n")
                for arg_name, arg_value in function_args.items():
                    value_to_log = arg_value[:50] if isinstance(arg_value, str) else str(arg_value)
                    log.write(f"Parameter {arg_name} = {value_to_log}\n")

                function_response = call_function(function_name, function_args)

                log.write(f"**Leaving agent {function_name}**\n")

                messages.append({
                    "role": "function",
                    "name": function_name,
                    "content": function_response,
                })
            else:
                break

        final_response = response.choices[0].message.content
        return final_response

# Input JSON
input_json_file = 'input.json'
with open(input_json_file, 'r') as f:
    input_json = json.load(f)

result = agent_loop(input_json)
print(result)

Average grade has been calculated and stored in 'query3.json'
The Python program to calculate the average grade has been successfully executed. The average grade has been calculated as 85.00625 and the result has been stored in 'query3.json'.
